In [72]:
import cv2
import glob
import numpy as np
DATA_DIR = "./input"

from __future__ import print_function, division
import os
import pandas as pd
import skimage
import random
import skimage.io
from skimage import transform
from skimage.viewer import ImageViewer
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
import glob


def read_list_from_file(list_file, comment='#', func=None):
    with open(list_file) as f:
        lines  = f.readlines()

    strings=[]
    for line in lines:
        s = line.split(comment, 1)[0].strip()
        if s != '':
            strings.append(s)
    if func is not None:
        strings=[func(s) for s in strings]

    return strings
        
def run_make_one_mask():

    split = 'names'
    ids = read_list_from_file(DATA_DIR + '/names/' + split, comment='#')

    num_ids = len(ids)
    for i in range(num_ids):
        id = ids[i]
        image_files = glob.glob(DATA_DIR + '/' + id + '/images/*.png')

        assert(len(image_files)==1)
        image_file=image_files[0]
        image = cv2.imread(image_file,cv2.IMREAD_COLOR)
        H,W,C = image.shape
        one_mask = np.zeros((H,W), dtype=bool)
        mask_files =glob.glob(DATA_DIR + '/' + id + '/masks/*.png')
        for mask_file in mask_files:
            mask = cv2.imread(mask_file,cv2.IMREAD_GRAYSCALE)
            one_mask = one_mask |(mask>128)

        one_mask = (one_mask*255).astype(np.uint8)
        cv2.imwrite(DATA_DIR + '/' + id + '/one_mask.png', one_mask)


In [73]:
run_make_one_mask()

In [202]:
def data_aug(image,label,angel=5,resize_rate=0.3, output_size=256):

    flip = random.randint(0, 1)
    size = image.shape[0]
    sh = random.random()/2-0.25
    rotate_angel = random.random()/180*np.pi*angel
    sigma = random.uniform(0,0.6)
    scaling = random.uniform(0.1,1)
    
    # Rescale
    transform.rescale(image, scaling, mode='reflect')
    transform.rescale(label, scaling, mode='reflect')   
        
    # Create affine transform
    afine_tf = transform.AffineTransform(shear=sh,rotation=rotate_angel)
    
    # Apply transform to image data
    image = transform.warp(image, inverse_map=afine_tf,mode='edge')
    label = transform.warp(label, inverse_map=afine_tf,mode='edge')
    
    # Ramdomly flip frame
    if flip:
        image = image[:,::-1,:]
        label = label[:,::-1]
        
    #Shuffle the RGB channels to get color invariance
    p = [0,1,2]
    random.shuffle(p)
    image[:,:,:] = image[:,:,p]    
    #Change exposure/intensity randomly:
    imageAlp = image[:,:,2]  
    imageRest = image[:,:,0:3]
    if random.randint(0,1)==0:
        #use clahe
        image = exposure.equalize_adapthist(image)    
    if random.randint(0,1)==0:
        image = exposure.rescale_intensity(image,(0,random.uniform(0.5,1)))        
    
    #Rotate the image
    rotate = random.randint(0,3)
    image = transform.rotate(image, rotate*np.pi/2)
    label = transform.rotate(label, rotate*np.pi/2)   
    #Corrupt data a little
    if sigma > 0:
        image = np.minimum(np.maximum(0.0,image + np.random.normal(0, sigma, image.shape)/255.0),255.0)
    
    
    # Randomly cropping image frame
    rsize = random.randint(np.floor(resize_rate*size),size)
    w_s = random.randint(0,size - rsize)
    h_s = random.randint(0,size - rsize)
    image = image[w_s:w_s+size,h_s:h_s+size,:]
    label = label[w_s:w_s+size,h_s:h_s+size]
    
    
    #Now we resize the image
    image = transform.resize(image, (output_size, output_size))
    label = transform.resize(label, (output_size, output_size))
    image = image.transpose(2, 1, 0)
    #label  = cv2.threshold(label, 128, 255, cv2.THRESH_BINARY)[1]
    label[label > 0.] = 1
    return image.astype(float), label.astype(float)


In [192]:
from torch.utils.data.dataset import Dataset

class ScienceDataset(Dataset):

    def __init__(self, split, mode='train', transform= None):
        super(ScienceDataset, self).__init__()
        self.split = split
        self.mode = mode
        self.transform = transform
        ids = read_list_from_file(DATA_DIR + '/names/' + split, comment='#')
        self.ids = ids

    def __getitem__(self, index):
        id   = self.ids[index]
        name = id.split('/')[-1]
        image_file = DATA_DIR + '/' + id + '/images/' + name +'.png'
        image = cv2.imread(image_file, cv2.IMREAD_COLOR)

        if self.mode in ['train']:
            mask_file =  DATA_DIR + '/' + id + '/one_mask.png'
            mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
            return self.transform(image,mask)

        if self.mode in ['test']:
            return self.transform(image)

    def __len__(self):
        return len(self.ids)

In [178]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *
def image_show(name, image, resize=1):
    H,W = image.shape[0:2]
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, image.astype(np.uint8))
    cv2.resizeWindow(name, round(resize*W), round(resize*H))

In [203]:
WIDTH = 256
HIGHT =256
def augment(image,mask):
        image = cv2.resize(image,(WIDTH,HIGHT))
        image = image.transpose(2, 1, 0)
        mask  = cv2.resize(mask,(WIDTH,HIGHT))
        mask  = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY)[1]
        mask[mask == 255.] = 1
        return image.astype(float),mask.astype(float)

dataset = ScienceDataset(
        'names', mode='train', transform = data_aug
    )
sampler = RandomSampler(dataset)

In [204]:
dataset[1][0]

/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


array([[[0.01698027, 0.0195621 , 0.02042393, ..., 0.02421154,
         0.02348771, 0.02170185],
        [0.01768505, 0.01276637, 0.01436983, ..., 0.02592269,
         0.02650966, 0.02126072],
        [0.0183383 , 0.02267278, 0.02272115, ..., 0.02395205,
         0.02474077, 0.0229647 ],
        ...,
        [0.02768914, 0.02957133, 0.03419073, ..., 0.04333054,
         0.05179265, 0.05471086],
        [0.02671863, 0.02858379, 0.0319523 , ..., 0.04411691,
         0.05128024, 0.05338923],
        [0.0241607 , 0.02785407, 0.03366561, ..., 0.04316877,
         0.05227152, 0.05395211]],

       [[0.01710708, 0.02015094, 0.01943791, ..., 0.02568571,
         0.02489939, 0.02241055],
        [0.01534537, 0.01025039, 0.01394382, ..., 0.0255337 ,
         0.02581012, 0.02226071],
        [0.01803845, 0.0207396 , 0.01956567, ..., 0.02520252,
         0.0259987 , 0.02470937],
        ...,
        [0.02643818, 0.0279955 , 0.03377537, ..., 0.04461747,
         0.05341933, 0.05370235],
        [0.0

In [116]:
import torch.nn as nn
def UpDownConv(in_channels, mid_channels, out_channels):
    return nn.Sequential(
            *ConvRelu(in_channels, mid_channels, kernel_size=3, stride=1, padding=1 ),
            *ConvRelu(mid_channels, out_channels, kernel_size=3, stride=1, padding=1 ),
    )
def ConvRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return [
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    ]
class UNet(nn.Module):
    
    def __init__(self, in_shape, num_classes):
        super(UNet, self).__init__()
        in_channels, height, width = in_shape

        self.down1 = nn.Sequential(
            *ConvRelu(in_channels, 16, kernel_size=3, stride=1, padding=1 ),
            *ConvRelu(16, 32, kernel_size=3, stride=2, padding=1 ),
        )
        #64
        self.down2 = UpDownConv(32, 64, 128)
        #32
        self.down3 = UpDownConv(128, 256, 512)
        #16
        self.down4 = UpDownConv(512, 512, 512)
        #8
        self.same = nn.Sequential(
            *ConvRelu(512,512, kernel_size=1, stride=1, padding=0 ),
        )
        #16
        self.up4 = UpDownConv(1024, 512, 512)
        #16
        self.up3 = UpDownConv(1024, 512, 128)
        #32
        self.up2 = UpDownConv(256, 128, 32)
        #64
        self.up1 = UpDownConv(64, 64, 32)
        #128

        self.up0 = nn.Sequential(
            *ConvRelu(32, 32, kernel_size=3, stride=1, padding=1 ),
        )
        #256

        self.classify = nn.Conv2d(32, num_classes, kernel_size=1, stride=1, padding=0 )

    def forward(self, x):

        down1 = self.down1(x)
        out   = F.max_pool2d(down1, kernel_size=2, stride=2) #64
        down2 = self.down2(out)
        out   = F.max_pool2d(down2, kernel_size=2, stride=2) #32
        down3 = self.down3(out)
        out   = F.max_pool2d(down3, kernel_size=2, stride=2) #16
        down4 = self.down4(out)
        out   = F.max_pool2d(down4, kernel_size=2, stride=2) # 8
        out   = self.same(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #16
        out   = torch.cat([down4, out],1)
        out   = self.up4(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #32
        out   = torch.cat([down3, out],1)
        out   = self.up3(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #64
        out   = torch.cat([down2, out],1)
        out   = self.up2(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #128
        out   = torch.cat([down1, out],1)
        out   = self.up1(out)
        out   = F.upsample(out, scale_factor=2, mode='bilinear') #256
        out   = self.up0(out)
        out   = self.classify(out)
        return out
    

In [118]:
num_classes = 1
batch_size  = 8
C = 3
from torch.autograd import Variable
import torch.nn.functional as F

inputs = torch.randn(batch_size,C,HIGHT,WIDTH)
labels = torch.LongTensor(batch_size,HIGHT,WIDTH).random_(1).type(torch.FloatTensor)

net = UNet(in_shape=(C,HIGHT,WIDTH), num_classes=1).train()
x = Variable(inputs)
y = Variable(labels)
logits = net.forward(x)

probs = F.sigmoid(logits)
probs_flat = probs.view (-1)
targets_flat = y.view(-1)

print('logits')
print(logits)
loss = nn.BCELoss()
output = loss(probs_flat, targets_flat)
output.backward()

print(type(net))
print(net)

logits
Variable containing:
( 0 , 0 ,.,.) = 
 -1.8262e-02  4.5847e-02 -3.8566e-03  ...   6.3177e-02  1.1645e-01  5.3935e-02
 -5.4142e-02 -3.4450e-02 -1.9488e-02  ...   1.5964e-01  1.6915e-01  6.3270e-02
  1.5994e-01  1.3885e-01  2.2499e-01  ...  -6.5189e-02  9.8443e-02  5.7684e-02
                 ...                   ⋱                   ...                
 -1.2149e-01 -9.1006e-02 -8.2508e-02  ...  -2.3114e-01 -6.0849e-02  1.6745e-01
 -8.6020e-02 -2.3827e-01 -2.6479e-01  ...  -1.5360e-01  3.7407e-03  1.2322e-01
  9.3998e-02 -1.3145e-01 -2.0767e-01  ...  -2.6521e-01 -2.1105e-01 -3.2904e-02
      ⋮  

( 1 , 0 ,.,.) = 
 -5.3981e-03  1.5119e-01  2.4639e-01  ...  -3.0234e-02  2.3453e-02  4.7593e-04
  4.7930e-02  6.8546e-03  1.5760e-01  ...   2.1348e-01  3.3936e-01  1.5158e-01
  9.8182e-02 -2.5223e-02  3.3031e-03  ...   1.9469e-01  2.4480e-01  1.1176e-01
                 ...                   ⋱                   ...                
  2.6013e-02  7.8945e-02 -2.0442e-02  ...   2.4072e-01  3.

In [119]:
output

Variable containing:
 0.7288
[torch.FloatTensor of size 1]

In [205]:
import torch.optim as optim
LR = 0.0002
batch_size = 5
net = UNet(in_shape=(C,HIGHT,WIDTH), num_classes=1).train()
epochs = 1
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
        
def train():
    optimizer = \
                optim.Adam(net.parameters(),
                           lr= LR, betas=(0.5, 0.999))
    train_dataset = ScienceDataset(
                                'names', mode='train',
                                transform = data_aug)
    train_loader  = DataLoader(
                        train_dataset,
                        sampler = RandomSampler(train_dataset),
                        batch_size  = batch_size,
                        drop_last   = True,
                        num_workers = 8)
    i = 0
    while  i < epochs:  
        sum_train_loss = 0.0
        sum_train_acc  = 0.0
        sum = 0
        net.train()
        optimizer.zero_grad()
        for index, data  in enumerate(train_loader):
            image,mask = data
            print(image.shape)
            tensors = Variable(image)
            masks   = Variable(mask)
            logits  = net(tensors.float())
            probs   = F.sigmoid(logits).view(-1)
            masks_flat = masks.view(-1)
            print(probs.shape)
            print(masks_flat)
            loss = nn.BCELoss()
            output = loss(probs, masks_flat.float())
            output.backward()
            optimizer.step()
            optimizer.zero_grad()
            adjust_learning_rate(optimizer, i)
            batch_loss = output.data[0]
            sum_train_loss += batch_loss
            print('\r %0.2f| %0.4f| %0.4f  %0.4f  ',i, LR, batch_loss, sum_train_loss)

        i = i +1


In [206]:
train()

/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/polinazablotskaia/anaconda/envs/py35/

torch.Size([5, 3, 256, 256])


/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


torch.Size([327680])
Variable containing:
 1
 1
 1
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6809605956077576 0.6809605956077576
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6722209453582764 1.353181540966034
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6502050161361694 2.0033865571022034
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6880558133125305 2.691442370414734
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6748801469802856 3.3663225173950195
t

torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6085004806518555 26.174000322818756
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 1
 1
 1
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6376951932907104 26.811695516109467
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6177183985710144 27.42941391468048
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.599848747253418 28.0292626619339
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.6289160251617432 28.658178687095642
tor

torch.Size([327680])
Variable containing:
 1
 1
 1
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.591711163520813 49.90935432910919
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.5830492973327637 50.492403626441956
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.5569607019424438 51.0493643283844
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.5755199193954468 51.624884247779846
torch.Size([5, 3, 256, 256])
torch.Size([327680])
Variable containing:
 0
 0
 0
⋮ 
 0
 0
 0
[torch.DoubleTensor of size 327680]

 %0.2f| %0.4f| %0.4f  %0.4f   0 0.0002 0.5476678013801575 52.172552049160004
tor

Process Process-33:
Process Process-37:
Process Process-40:
Process Process-34:
Process Process-35:
Process Process-36:
Process Process-38:
Process Process-39:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/polinazablotskaia/anaconda/envs/py35/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/polinazablotskaia/anaconda/envs/py35/lib/pyt

KeyboardInterrupt: 

In [127]:
import torch 
torch.save(net.state_dict(), DATA_DIR +'/checkpoint/model.pth')